# Filtering CORINE Land Cover 2018 dataset for Germany

The CORINE Land Cover 2018 consists of an inventory of land cover in 44 classes based on Sentinel-2 and Landsat-8 data. It can be accessed here: https://land.copernicus.eu/pan-european/corine-land-cover

In this notebook, the areas of Germany classified as *wetland* (Code 4xx) are extracted from the Corine Land Cover 2018 dataset for further processing.

In [30]:
import numpy as np
import pandas as pd
import geopandas

This notebook assumes that the data is made available under the following paths. The CLC 2018 data must be vector-based and experiments have been performed with the official distribution from the Copernicus programme in the ESRI Geodatabase format. But every other geodata format supported by geopandas should work.

In [ ]:
PATH_TO_NUTS_2021_LVL_3 = "../eurostat/NUTS_RG_01M_2021_4326_LEVL_3.json"
PATH_TO_CLC2018 = "u2018_clc2018_v2020_20u1_fgdb/DATA/U2018_CLC2018_V2020_20u1.gdb"

Utilizing NUTS Level 3 administrative zones as mask for fetching the CLC2018 data of Germany.

In [31]:
nuts_level_3 = geopandas.read_file(PATH_TO_NUTS_2021_LVL_3)
nuts_level_3 = nuts_level_3.set_crs("EPSG:4326")

In [32]:
nuts_level_3_germany = nuts_level_3[nuts_level_3["CNTR_CODE"] == "DE"]

In [33]:
clc2018_germany = geopandas.read_file(PATH_TO_CLC2018, mask=nuts_level_3_germany)

In [34]:
clc2018_germany = clc2018_germany.to_crs("EPSG:4326")

Extracting the three relevant ecosystems from the dataset: 

- "Inland marshes" (411) 
- "Peatbogs" (412)
- "Salt marshes" (421)

Intertidal flats (423) are excluded as they are not in the scope of this project. See the [CORINE Land Cover nomenclature guidelines](https://land.copernicus.eu/user-corner/technical-library/corine-land-cover-nomenclature-guidelines/html/index.html) for further information about the classification.

In [35]:
wetlands = clc2018_germany[clc2018_germany["Code_18"].isin(["411", "412", "421"])]

In [36]:
wetlands

,Code_18,Remark,Area_Ha,ID,Shape_Length,Shape_Area,geometry
10086,411,NaN,28.618544,EU_595529,3837.363612,2.861854e+05,"MULTIPOLYGON (((6.76445 49.85424, 6.76410 49.8..."
10087,411,NaN,65.836005,EU_595672,6184.452802,6.583601e+05,"MULTIPOLYGON (((6.10114 51.25879, 6.10007 51.2..."
10088,412,NaN,187.224936,EU_595740,7536.893593,1.872249e+06,"MULTIPOLYGON (((6.25120 50.63017, 6.25132 50.6..."
10241,411,NaN,157.419666,EU_596621,14376.362389,1.574197e+06,"MULTIPOLYGON (((6.65722 49.25293, 6.65781 49.2..."
11440,411,NaN,33.962090,EU_1029312,3174.334137,3.396209e+05,"MULTIPOLYGON (((9.66701 47.58284, 9.66556 47.5..."
...,...,...,...,...,...,...,...
169505,412,NaN,41.769345,EU_2370116,5376.531991,4.176934e+05,"MULTIPOLYGON (((12.91782 48.21419, 12.91978 48..."
169966,411,NaN,43.222356,EU_2371742,3878.241606,4.322236e+05,"MULTIPOLYGON (((14.38708 51.63956, 14.38614 51..."
170041,412,NaN,63.587904,EU_2372303,4994.571340,6.358790e+05,"MULTIPOLYGON (((7.01510 52.61229, 7.01641 52.6..."
170042,412,NaN,2009.372572,EU_2372304,37268.415609,2.009373e+07,"MULTIPOLYGON (((7.07941 52.70460, 7.07962 52.7..."


Store the resulting geodataframe as shapefile for later use

In [37]:
wetlands[["Code_18", "Area_Ha", "ID", "geometry"]].to_file("clc2018_germany_wetlands.shp")